In [1]:
from utils import get_train_str, get_train_and_eval_data, get_eval_preds, train_prompt

import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jerryliu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load up the titanic data
train_df, train_labels, eval_df, eval_labels = get_train_and_eval_data("data/train.csv")

## Few-shot Prompting with GPT-3 for Titanic Dataset
In this section, we can show how we can prompt GPT-3 on its own (without using GPT Index) to attain ~80% accuracy on Titanic! 

We can do this by simply providing a few example inputs. Or we can simply provide no example inputs at all (zero-shot). Both achieve the same results.

In [3]:
# first demonstrate the prompt template
print(train_prompt.template)

The following structured data is provided in "Feature Name":"Feature Value" format.
Each datapoint describes a passenger on the Titanic.
The task is to decide whether the passenger survived.
Some example datapoints are given below: 
-------------------
{train_str}
-------------------
Given this, predict whether the following passenger survived. Return answer as a number between 0 or 1. 
{eval_str}
Survived: 


In [4]:
# Get "training" prompt string
train_n = 10
eval_n = 40
train_str = get_train_str(train_df, train_labels, train_n=train_n)
print(f"Example datapoints in `train_str`: \n{train_str}")

Example datapoints in `train_str`: 
This is the Data:
Age:28.0
Embarked:S
Fare:7.8958
Parch:0
Pclass:3
Sex:male
SibSp:0
This is the correct answer:
Survived: 0

This is the Data:
Age:17.0
Embarked:S
Fare:7.925
Parch:2
Pclass:3
Sex:female
SibSp:4
This is the correct answer:
Survived: 1

This is the Data:
Age:30.0
Embarked:S
Fare:16.1
Parch:0
Pclass:3
Sex:male
SibSp:1
This is the correct answer:
Survived: 0

This is the Data:
Age:22.0
Embarked:S
Fare:7.25
Parch:0
Pclass:3
Sex:male
SibSp:0
This is the correct answer:
Survived: 0

This is the Data:
Age:45.0
Embarked:S
Fare:13.5
Parch:0
Pclass:2
Sex:female
SibSp:0
This is the correct answer:
Survived: 1

This is the Data:
Age:25.0
Embarked:S
Fare:0.0
Parch:0
Pclass:3
Sex:male
SibSp:0
This is the correct answer:
Survived: 1

This is the Data:
Age:18.0
Embarked:S
Fare:20.2125
Parch:1
Pclass:3
Sex:male
SibSp:1
This is the correct answer:
Survived: 0

This is the Data:
Age:33.0
Embarked:S
Fare:9.5
Parch:0
Pclass:3
Sex:male
SibSp:0
This is the c

#### Do evaluation with the training prompt string

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

eval_preds = get_eval_preds(train_prompt, train_str, eval_df, n=eval_n)
eval_label_chunk = eval_labels[:eval_n]

In [7]:
acc = accuracy_score(eval_label_chunk, np.array(eval_preds).round())
print(f"ACCURACY: {acc}")

ACCURACY: 0.8


#### Do evaluation with no training prompt string! 

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

eval_preds_null = get_eval_preds(train_prompt, "", eval_df, n=eval_n)
eval_label_chunk = eval_labels[:eval_n]

In [11]:
acc_null = accuracy_score(eval_label_chunk, np.array(eval_preds_null).round())
print(f"ACCURACY: {acc_null}")

ACCURACY: 0.8


## Extending with Summary Index

#### Build Index

In [12]:
from llama_index import SummaryIndex
from llama_index.schema import Document

In [13]:
index = SummaryIndex([])

In [ ]:
# insertion into index
batch_size = 40
num_train_chunks = 5

for i in range(num_train_chunks):
    print(f"Inserting chunk: {i}/{num_train_chunks}")
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size
    train_batch = train_df.iloc[start_idx : end_idx + batch_size]
    labels_batch = train_labels.iloc[start_idx : end_idx + batch_size]
    all_train_str = get_train_str(train_batch, labels_batch, train_n=batch_size)
    index.insert(Document(text=all_train_str))

#### Query Index

In [15]:
from utils import query_str, qa_data_prompt, refine_prompt

In [16]:
query_str

'Which is the relationship between these features and predicting survival?'

In [17]:
query_engine = index.as_query_engine(
    text_qa_template=qa_data_prompt,
    refine_template=refine_prompt,
)
response = query_engine.query(
    query_str,
)

> Starting query: Which is the relationship between these features and predicting survival?


In [19]:
print(response)



There is no definitive answer to this question, as the relationship between the features and predicting survival will vary depending on the data. However, some possible relationships include: age (younger passengers are more likely to survive), sex (females are more likely to survive), fare (passengers who paid more for their ticket are more likely to survive), and pclass (passengers in first or second class are more likely to survive).


#### Get Predictions and Evaluate

In [26]:
# get eval preds
from utils import train_prompt_with_context

train_str = response
print(train_prompt_with_context.template)
print(f"\n\n`train_str`: {train_str}")

The following structured data is provided in "Feature Name":"Feature Value" format.
Each datapoint describes a passenger on the Titanic.
The task is to decide whether the passenger survived.
We discovered the following relationship between features and survival:
-------------------
{train_str}
-------------------
Given this, predict whether the following passenger survived. 
Return answer as a number between 0 or 1. 
{eval_str}
Survived: 


`train_str`: 

There is no definitive answer to this question, as the relationship between the features and predicting survival will vary depending on the data. However, some possible relationships include: age (younger passengers are more likely to survive), sex (females are more likely to survive), fare (passengers who paid more for their ticket are more likely to survive), and pclass (passengers in first or second class are more likely to survive).


In [ ]:
# do evaluation
from sklearn.metrics import accuracy_score
import numpy as np

eval_n = 40
eval_preds = get_eval_preds(train_prompt_with_context, train_str, eval_df, n=eval_n)

In [28]:
eval_label_chunk = eval_labels[:eval_n]
acc = accuracy_score(eval_label_chunk, np.array(eval_preds).round())
print(f"ACCURACY: {acc}")

ACCURACY: 0.85
